In [2]:
import pandas as pd

In [3]:
data = pd.read_csv("/home/m7md/Desktop/Data_Mining_project/DataSets/DataSetForRegression/House_Rent_Dataset_with_Missing_Values.csv")
data.head()

,Posted On,BHK,Rent,Size,Floor,Area Type,Area Locality,City,Furnishing Status,Tenant Preferred,Bathroom,Point of Contact
0,2022-05-18,2.0,10000.0,1100,Ground out of 2,Super Area,Bandel,Kolkata,Unfurnished,Bachelors/Family,2,Contact Owner
1,2022-05-13,2.0,20000.0,800,1 out of 3,Super Area,"Phool Bagan, Kankurgachi",Kolkata,Semi-Furnished,Bachelors/Family,1,Contact Owner
2,2022-05-16,2.0,NaN,1000,1 out of 3,Super Area,Salt Lake City Sector 2,Kolkata,Semi-Furnished,Bachelors/Family,1,Contact Owner
3,2022-07-04,2.0,10000.0,800,1 out of 2,Super Area,Dumdum Park,Kolkata,Unfurnished,Bachelors/Family,1,Contact Owner
4,2022-05-09,2.0,7500.0,850,1 out of 2,Carpet Area,South Dum Dum,Kolkata,Unfurnished,Bachelors,1,Contact Owner


In [4]:
null_count = data.isnull().sum().sum()
null_count

948

In [5]:
data.shape

(4746, 12)

In [6]:
data = pd.read_csv("/home/m7md/Desktop/Data_Mining_project/DataSets/DataSetForRegression/House_Rent_Dataset_with_Missing_Values.csv")

In [7]:
from sklearn.preprocessing import MinMaxScaler

# Step 1: Handle Missing Values
# Checking the percentage of missing values in each column
missing_values = data.isnull().mean() 

# Handling missing values in BHK and Rent (simple imputation for now)
# Assuming mean imputation for numerical missing values
data['BHK'] = data['BHK'].fillna(data['BHK'].mean())
data['Rent'] = data['Rent'].fillna(data['Rent'].mean())

In [8]:
null_count = data.isnull().sum().sum()
null_count

0

In [9]:
# Step 2: Process the "Floor" column
# Extract the current floor and total floors as separate numeric columns
floor_split = data['Floor'].str.extract(r'(\d+|Ground|Upper) out of (\d+|Unknown)')

# Replace textual entries with numeric equivalents
floor_split[0].replace({'Ground': 0, 'Upper': None}, inplace=True)
floor_split[1].replace({'Unknown': None}, inplace=True)

# Convert the split data to numeric types
data['Current Floor'] = pd.to_numeric(floor_split[0], errors='coerce')
data['Total Floors'] = pd.to_numeric(floor_split[1], errors='coerce')

# Drop the original "Floor" column
data.drop(columns=['Floor'], inplace=True)

/tmp/ipykernel_64495/893045906.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  floor_split[0].replace({'Ground': 0, 'Upper': None}, inplace=True)
/tmp/ipykernel_64495/893045906.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, in

In [10]:
print(data.columns)

Index(['Posted On', 'BHK', 'Rent', 'Size', 'Area Type', 'Area Locality',
       'City', 'Furnishing Status', 'Tenant Preferred', 'Bathroom',
       'Point of Contact', 'Current Floor', 'Total Floors'],
      dtype='object')


In [11]:
data.shape

(4746, 13)

In [12]:
# Step 3: Encode categorical variables
# One-hot encode categorical columns
categorical_columns = ['Area Type', 'City', 'Furnishing Status', 'Tenant Preferred', 'Point of Contact']
data = pd.get_dummies(data, columns=categorical_columns, drop_first=True)

# Display the processed dataset info and preview
data.info(), data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4746 entries, 0 to 4745
Data columns (total 21 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Posted On                          4746 non-null   object 
 1   BHK                                4746 non-null   float64
 2   Rent                               4746 non-null   float64
 3   Size                               4746 non-null   int64  
 4   Area Locality                      4746 non-null   object 
 5   Bathroom                           4746 non-null   int64  
 6   Current Floor                      4708 non-null   float64
 7   Total Floors                       4708 non-null   float64
 8   Area Type_Carpet Area              4746 non-null   bool   
 9   Area Type_Super Area               4746 non-null   bool   
 10  City_Chennai                       4746 non-null   bool   
 11  City_Delhi                         4746 non-null   bool 

(None,
     Posted On  BHK          Rent  Size             Area Locality  Bathroom  \
 0  2022-05-18  2.0  10000.000000  1100                    Bandel         2   
 1  2022-05-13  2.0  20000.000000   800  Phool Bagan, Kankurgachi         1   
 2  2022-05-16  2.0  34970.323736  1000   Salt Lake City Sector 2         1   
 3  2022-07-04  2.0  10000.000000   800               Dumdum Park         1   
 4  2022-05-09  2.0   7500.000000   850             South Dum Dum         1   
 
    Current Floor  Total Floors  Area Type_Carpet Area  Area Type_Super Area  \
 0            0.0           2.0                  False                  True   
 1            1.0           3.0                  False                  True   
 2            1.0           3.0                  False                  True   
 3            1.0           2.0                  False                  True   
 4            1.0           2.0                   True                 False   
 
    ...  City_Delhi  City_Hyderabad

In [13]:
# Step 4: Address missing values in "Current Floor" and "Total Floors"
# Impute with median since floors are ordinal/numeric
data['Current Floor'] = data['Current Floor'].fillna(data['Current Floor'].median())
data['Total Floors'] = data['Total Floors'].fillna(data['Total Floors'].median())

In [14]:
# Step 5: Normalize numerical columns
# Identify numerical columns for normalization
numerical_columns = ['BHK', 'Rent', 'Size', 'Bathroom', 'Current Floor', 'Total Floors']

In [15]:
# Apply Min-Max Scaling
scaler = MinMaxScaler()
data[numerical_columns] = scaler.fit_transform(data[numerical_columns])

In [16]:
# Convert "Posted On" to a more usable format (if needed for time-series analysis)
data['Posted On'] = pd.to_datetime(data['Posted On'])

In [17]:
# Dropping "Area Locality" for simplicity since it has too many unique values for encoding
data.drop(columns=['Area Locality'], inplace=True)

In [18]:
# Final dataset info and preview
data.info(), data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4746 entries, 0 to 4745
Data columns (total 20 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   Posted On                          4746 non-null   datetime64[ns]
 1   BHK                                4746 non-null   float64       
 2   Rent                               4746 non-null   float64       
 3   Size                               4746 non-null   float64       
 4   Bathroom                           4746 non-null   float64       
 5   Current Floor                      4746 non-null   float64       
 6   Total Floors                       4746 non-null   float64       
 7   Area Type_Carpet Area              4746 non-null   bool          
 8   Area Type_Super Area               4746 non-null   bool          
 9   City_Chennai                       4746 non-null   bool          
 10  City_Delhi                         4

(None,
    Posted On  BHK      Rent      Size  Bathroom  Current Floor  Total Floors  \
 0 2022-05-18  0.2  0.002515  0.136421  0.111111       0.000000      0.011364   
 1 2022-05-13  0.2  0.005373  0.098874  0.000000       0.013158      0.022727   
 2 2022-05-16  0.2  0.009652  0.123905  0.000000       0.013158      0.022727   
 3 2022-07-04  0.2  0.002515  0.098874  0.000000       0.013158      0.011364   
 4 2022-05-09  0.2  0.001801  0.105131  0.000000       0.013158      0.011364   
 
    Area Type_Carpet Area  Area Type_Super Area  City_Chennai  City_Delhi  \
 0                  False                  True         False       False   
 1                  False                  True         False       False   
 2                  False                  True         False       False   
 3                  False                  True         False       False   
 4                   True                 False         False       False   
 
    City_Hyderabad  City_Kolkata  City_Mu

In [19]:
data.to_csv("/home/m7md/Desktop/Data_Mining_project/DataSets/DataSetForRegression/output/Cleaned_House_Rent_Dataset_with_Missing_Values.csv")